In [1]:
import itertools
import pyspark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
803,application_1551670200070_0007,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
spark.stop()

In [3]:
sc=pyspark.SparkContext("local","Apriori")

In [4]:
text_input = sc.textFile("browsing.txt")

In [5]:
from operator import add
no_of_baskets = text_input.map(lambda line: line.split()).count()
no_of_baskets

31101

In [6]:
txt_RDD = text_input.map(lambda line:line.split())
txt_RDD.take(10)

[['FRO11987', 'ELE17451', 'ELE89019', 'SNA90258', 'GRO99222'], ['GRO99222', 'GRO12298', 'FRO12685', 'ELE91550', 'SNA11465', 'ELE26917', 'ELE52966', 'FRO90334', 'SNA30755', 'ELE17451', 'FRO84225', 'SNA80192'], ['ELE17451', 'GRO73461', 'DAI22896', 'SNA99873', 'FRO86643'], ['ELE17451', 'ELE37798', 'FRO86643', 'GRO56989', 'ELE23393', 'SNA11465'], ['ELE17451', 'SNA69641', 'FRO86643', 'FRO78087', 'SNA11465', 'GRO39357', 'ELE28573', 'ELE11375', 'DAI54444'], ['ELE17451', 'GRO73461', 'DAI22896', 'SNA99873', 'FRO18919', 'DAI50921', 'SNA80192', 'GRO75578'], ['ELE17451', 'ELE59935', 'FRO18919', 'ELE23393', 'SNA80192', 'SNA85662', 'SNA91554', 'DAI22177'], ['ELE17451', 'SNA69641', 'FRO18919', 'SNA90258', 'ELE28573', 'ELE11375', 'DAI14125', 'FRO78087'], ['ELE17451', 'GRO73461', 'DAI22896', 'SNA80192', 'SNA85662', 'SNA90258', 'DAI46755', 'FRO81176', 'ELE66810', 'DAI49199', 'DAI91535', 'GRO94758', 'ELE94711', 'DAI22177'], ['ELE17451', 'SNA69641', 'DAI91535', 'GRO94758', 'GRO99222', 'FRO76833', 'FRO8117

In [7]:
individual_counts = text_input.flatMap(lambda line: line.split()).map(lambda x: (x,1)).reduceByKey(add)
individual_counts.take(10)

[('FRO11987', 104), ('ELE17451', 3875), ('ELE89019', 38), ('SNA90258', 550), ('GRO99222', 906), ('GRO12298', 385), ('FRO12685', 23), ('ELE91550', 23), ('SNA11465', 142), ('ELE26917', 2292)]

In [8]:
freq_items_counts = individual_counts.filter(lambda x: x[1]>=100).sortBy(lambda x: x[0])
freq_items_counts.take(10)

[('DAI11223', 155), ('DAI11778', 117), ('DAI13194', 116), ('DAI13266', 181), ('DAI13788', 213), ('DAI13902', 233), ('DAI14125', 163), ('DAI14181', 110), ('DAI14470', 153), ('DAI15681', 132)]

In [9]:
freq_items_counts_list = individual_counts.filter(lambda x: x[1]>=100).sortBy(lambda x: x[0]).collect()

In [10]:
freq_items = freq_items_counts.map(lambda x: x[0])
freq_items.take(10)

['DAI11223', 'DAI11778', 'DAI13194', 'DAI13266', 'DAI13788', 'DAI13902', 'DAI14125', 'DAI14181', 'DAI14470', 'DAI15681']

In [11]:
freq_items_list = freq_items_counts.map(lambda x: x[0]).collect()

In [12]:
freq_items_list[:10]

['DAI11223', 'DAI11778', 'DAI13194', 'DAI13266', 'DAI13788', 'DAI13902', 'DAI14125', 'DAI14181', 'DAI14470', 'DAI15681']

In [13]:
no_of_freq_items=freq_items.count()
no_of_freq_items

647

In [14]:
def filter_and_pair(lst):
    ret=[]
    pairs=[]
    for i in lst:
        if i in freq_items_list:
            ret.append(i)
    ret.sort()
    for pair in itertools.combinations(ret,2):
        pairs.append((pair,1))
    return(pairs)

In [15]:
def filter_2(lst):
    ret=[]
    trips=[]
    for i in lst:
        if i in freq_items_2:
            ret.append(i)
    ret.sort()
    for trip in itertools.combinations(ret,3):
        trips.append(((trip),1))
    return(trips)

In [16]:
def confidence(elem):
    lst=[]
    conf1 = elem[1]/(freq_pair_counts.filter(lambda x: x[0]==elem[0][0]).collect())[0][1]
    conf2 = elem[1]/(freq_items_counts.filter(lambda x: x[0]==elem[0][1]).collect())[0][1]
    lst.append(((elem[0],elem[1]),conf1))
    lst.append(((elem[1],elem[0]),conf2))
    return(lst)
    

In [17]:
def confidence2(elem):
    lii=[]
    for i in (itertools.combinations(elem[0],2)):
        conf=elem[1]/(freq_pair_counts.filter(lambda X: x[0]==i)).collect[0][1]
        lii.append((i,conf))
    ret(lii)

In [18]:
pair_counts = txt_RDD.flatMap(lambda x: filter_and_pair(x)).reduceByKey(add).sortBy( lambda x: x[0])
pair_counts.take(10)

[(('DAI11223', 'DAI13194'), 1), (('DAI11223', 'DAI13266'), 1), (('DAI11223', 'DAI13902'), 2), (('DAI11223', 'DAI20585'), 1), (('DAI11223', 'DAI22177'), 17), (('DAI11223', 'DAI22240'), 7), (('DAI11223', 'DAI22534'), 1), (('DAI11223', 'DAI22896'), 7), (('DAI11223', 'DAI25796'), 1), (('DAI11223', 'DAI26369'), 3)]

In [19]:
freq_pair_counts = pair_counts.filter(lambda x: x[1]>=100)
freq_pair_counts.take(10)
freq_pair_counts_list = freq_pair_counts.collect()
freq_pair_counts_list[:10]


[(('DAI16732', 'FRO78087'), 106), (('DAI18527', 'SNA44451'), 102), (('DAI22177', 'DAI31081'), 127), (('DAI22177', 'DAI62779'), 382), (('DAI22177', 'DAI63921'), 136), (('DAI22177', 'DAI75645'), 123), (('DAI22177', 'DAI83733'), 126), (('DAI22177', 'DAI85309'), 172), (('DAI22177', 'ELE17451'), 203), (('DAI22177', 'ELE26917'), 134)]

In [20]:
freq_pairs = freq_pair_counts.map(lambda x: x[0])
freq_pairs.take(10)

[('DAI16732', 'FRO78087'), ('DAI18527', 'SNA44451'), ('DAI22177', 'DAI31081'), ('DAI22177', 'DAI62779'), ('DAI22177', 'DAI63921'), ('DAI22177', 'DAI75645'), ('DAI22177', 'DAI83733'), ('DAI22177', 'DAI85309'), ('DAI22177', 'ELE17451'), ('DAI22177', 'ELE26917')]

In [21]:
conf_lst=[]
for i in freq_pair_counts_list:
    conf1 = i[1]/(freq_items_counts.filter(lambda x: x[0]==i[0][0]).collect())[0][1]
    conf2 = i[1]/(freq_items_counts.filter(lambda x: x[0]==i[0][1]).collect())[0][1]
    conf_lst.append(((i[0][0],i[0][1]),conf1))
    conf_lst.append(((i[0][1],i[0][0]),conf2))

In [22]:
conf_lst.sort(key=lambda x: -x[1])
conf_lst[:5]

[(('DAI93865', 'FRO40251'), 1.0), (('GRO85051', 'FRO40251'), 0.999176276771005), (('GRO38636', 'FRO40251'), 0.9906542056074766), (('ELE12951', 'FRO40251'), 0.9905660377358491), (('DAI88079', 'FRO40251'), 0.9867256637168141)]

In [23]:
freq_items_2 = freq_pairs.flatMap(lambda x: x).collect()
freq_items_2[:10]

['DAI16732', 'FRO78087', 'DAI18527', 'SNA44451', 'DAI22177', 'DAI31081', 'DAI22177', 'DAI62779', 'DAI22177', 'DAI63921']

In [24]:
trips_counts = txt_RDD.flatMap(lambda x: filter_2(x)).reduceByKey(add).sortBy(lambda x: x[0])
trips_counts.take(10)

[(('DAI16732', 'DAI18527', 'DAI35347'), 2), (('DAI16732', 'DAI18527', 'DAI59508'), 2), (('DAI16732', 'DAI18527', 'DAI62779'), 1), (('DAI16732', 'DAI18527', 'DAI63921'), 2), (('DAI16732', 'DAI18527', 'DAI85309'), 1), (('DAI16732', 'DAI18527', 'DAI95741'), 1), (('DAI16732', 'DAI18527', 'ELE12792'), 1), (('DAI16732', 'DAI18527', 'ELE12845'), 1), (('DAI16732', 'DAI18527', 'ELE20398'), 1), (('DAI16732', 'DAI18527', 'ELE21353'), 3)]

In [25]:
freq_trips = trips_counts.filter(lambda x: x[1]>=100)
freq_trips.take(10)

[(('DAI22896', 'DAI62779', 'GRO73461'), 101), (('DAI23334', 'DAI62779', 'ELE92920'), 143), (('DAI31081', 'DAI62779', 'ELE17451'), 103), (('DAI31081', 'DAI75645', 'FRO40251'), 122), (('DAI31081', 'ELE32164', 'GRO59710'), 112), (('DAI31081', 'FRO40251', 'GRO85051'), 102), (('DAI31081', 'FRO40251', 'SNA80324'), 103), (('DAI42083', 'DAI62779', 'DAI92600'), 105), (('DAI42083', 'DAI92600', 'ELE17451'), 117), (('DAI42493', 'DAI62779', 'ELE17451'), 112)]

In [26]:
freq_trips_list = trips_counts.filter(lambda x: x[1]>=100).collect()
conf_trips=[]
for i in freq_trips_list:
    a=set(i[0])
    for j in (itertools.combinations(i[0],2)):
        b=set(j)
        c=list(a-b)
        conf=i[1]/(freq_pair_counts.filter(lambda x: x[0]==j)).collect()[0][1]
        conf_trips.append(((j,c[0]),conf))

In [27]:
conf_trips.sort(key = lambda x: -x[1])

In [28]:
conf_trips[:5]

[((('DAI23334', 'ELE92920'), 'DAI62779'), 1.0), ((('DAI31081', 'GRO85051'), 'FRO40251'), 1.0), ((('DAI55911', 'GRO85051'), 'FRO40251'), 1.0), ((('DAI62779', 'DAI88079'), 'FRO40251'), 1.0), ((('DAI75645', 'GRO85051'), 'FRO40251'), 1.0)]